In [18]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/JKJ1A
data/  LICENSE  model/  notebook/  README.md  src/


# Day1補足資料

Pytorchに苦戦している人が多そうなので，その補足としてこの資料を書いています．

Day1ではブラックボックスで説明しても意味不明だろうな，と思いいろいろ細かく書き，また説明しました．そのせいでむしろ混乱した人もいるかもしれません．

ここではあえてブラックボックスとして書いてみます（`src/`にある`.py`ファイルを利用します）．

In [19]:
# 三要素の１：データの準備
from src.cifar10 import load_data                   # src/cifar10.pyを利用
trainloader, testloader, classes = load_data(128)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
# 三要素の２：モデルの準備
from src.vgg import VGG as Net      # src/vgg.pyを利用
net = Net('VGG11')
net.cuda();  # GPU

In [21]:
# 三要素の３：学習

# 1エポック分の操作．
def train_step(net, dataloader, optimizer, criterion):
    for x, y in dataloader:
        x, y = x.cuda(), y.cuda()
        
        optimizer.zero_grad()

        output = net(x)
        loss = criterion(output, y)  # 誤差の計算

        loss.backward()              # 勾配の計算
        optimizer.step()             # 勾配法（SGD）を行う


# 上の1エポック分の操作を繰り返す
def train(net, dataloader, optimizer, criterion, nepochs):
    for epoch in range(nepochs):
        print(f'epoch {epoch}')
        train_step(net, trainloader, optimizer, criterion)

In [22]:
%%time
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()                 # 誤差関数をセット
optimizer = optim.SGD(net.parameters(), lr=0.01)  # 確率的勾配法をセット（引数にモデルのパラメタを与えてるのに注意）

nepochs = 2
train(net, trainloader, optimizer, criterion, nepochs)

epoch 0
epoch 1
CPU times: user 58.8 s, sys: 493 ms, total: 59.3 s
Wall time: 59.8 s


上では簡素にするためにプリントなどもしてないので，ロスが下がっているかわかりません．少しだけprintを追加したのが以下です．

In [23]:
# 三要素の３：学習

# 1エポック分の操作．
def train_step(net, dataloader, optimizer, criterion):

    total_loss = 0.0

    for x, y in dataloader:
        x, y = x.cuda(), y.cuda()
        
        optimizer.zero_grad()

        output = net(x)
        loss = criterion(output, y)  # 誤差の計算

        loss.backward()              # 勾配の計算
        optimizer.step()             # 勾配法（SGD）を行う

        total_loss += loss

    avg_loss = total_loss / len(dataloader)  # 平均にする

    return avg_loss


# 上の1エポック分の操作を繰り返す
def train(net, dataloader, optimizer, criterion, nepochs):
    for epoch in range(nepochs):
        loss = train_step(net, trainloader, optimizer, criterion)
        print(f'epoch {epoch}: {loss:.4f}')

### 結局全体像は以下の通り

- データセットを変えたいなら，`load_data`の中身を変える
- モデルを変えるなら，`Net`の中身を変える
- 訓練方法を変えるなら，`train`（特に`train_step`）を変える

In [24]:
%%time
# データのロード
trainloader, testloader, classes = load_data(128)

# モデルの準備
net = Net('VGG11').cuda()

criterion = nn.CrossEntropyLoss()                 # 誤差関数
optimizer = optim.SGD(net.parameters(), lr=0.01)  # 確率的勾配法

# 最適化
nepochs = 2
train(net, trainloader, optimizer, criterion, nepochs)

Files already downloaded and verified
Files already downloaded and verified
epoch 0: 1.4965
epoch 1: 1.0567
CPU times: user 1min, sys: 1.16 s, total: 1min 1s
Wall time: 1min 3s
